In [18]:
import os
from icrawler.builtin import BingImageCrawler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

print("Please upload your 'test.jpg' file.")
uploaded = files.upload()

if 'test.jpg' in uploaded:
  print(f"'test.jpg' successfully uploaded with length {len(uploaded['test.jpg'])} bytes")
else:
  print("Warning: 'test.jpg' was not found among the uploaded files. Please ensure you upload the correct file.")

animals = ['cat','dog','lion','elephant']

base_dir = 'dataset'
train_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for animal in animals:
    train_path = os.path.join(train_dir,animal)
    val_path = os.path.join(val_dir,animal)
    os.makedirs(train_path, exist_ok=True)
    os.makedirs(val_path, exist_ok=True)
    crawler = BingImageCrawler(storage={'root_dir':train_path})
    crawler.crawl(keyword=animal, max_num=40)
    crawler = BingImageCrawler(storage={'root_dir':val_path})
    crawler.crawl(keyword=animal, max_num=10)

img_size = (150,150)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

model = models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes,activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_generator,epochs=5,validation_data=val_generator)

loss,accuracy=model.evaluate(val_generator)
print('Validation Accuracy:',accuracy)

model.save('animal_cnn_model.h5')

img=image.load_img('test.jpg',target_size=img_size)
img_array=image.img_to_array(img)/255.0
img_array=np.expand_dims(img_array,axis=0)

prediction=model.predict(img_array)
class_indices=train_generator.class_indices
labels=dict((v,k) for k,v in class_indices.items())

print('Predicted Animal:',labels[np.argmax(prediction)])

Please upload your 'test.jpg' file.


Saving test.jpg to test (1).jpg
Found 160 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.2674 - loss: 1.7950 - val_accuracy: 0.2500 - val_loss: 1.3704
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.2998 - loss: 1.3703 - val_accuracy: 0.3250 - val_loss: 1.3536
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.3024 - loss: 1.3269 - val_accuracy: 0.5500 - val_loss: 1.2472
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.4734 - loss: 1.2383 - val_accuracy: 0.5250 - val_loss: 1.1375
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.4941 - loss: 1.1974 - val_accuracy: 0.4750 - val_loss: 1.1677
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.4729 - loss: 1.1640


Validation Accuracy: 0.4749999940395355
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Predicted Animal: cat
